# *In silico* color mixing and optimization

This is the first full color mixing exercise in the course Autonomous materials discovery (47332). 

In this exercise you will explore the `SilicoColorMixer` python class and get familiarity with common optimizers. You should by now know to basic principles of optimizers and some standard ones.

The reason for using the `SilicoColorMixer` in this course is that you can experiment with your code without having to wait for the LEGO robot. You should thus experience, that code produced in this exercise will be very useful and easy to transfer to later exercises involving the robot. Towards the end of the course, you will also use the *in silico* model in combination with the LEGO robot for improved performance. 

## Exercise 1: Get familiar with `SilicoColorMixer`

Before getting ahead of yourselves, let us look at the `SilicoColorMixer`. Start by importing it.

In [ ]:
from silico_color_mixer import SilicoColorMixer

Let us see the description of the `SilicoColorMixer` class. Place your cursor in the above cell and press `Shift`+`Tab`. This will open a new window below the cell showing you some of the possible inputs and default parameters for the class. You can see all possible inputs and the Docstring, which describes the class, by clicking on the plus or up-arrow in the corner of the window. The output will look something like this:

Lets go over the possible inputs one by one:

* noise: This can either be `False` or a dictionary like the default one and must contain the entries `'color'`, `'volume'`, and `'measurement'`. You can read about each of them in the Docstring above.
* colors: A list with the names of the colors we will be using. To be transferable to the LEGO robot, this list should contain 4 colors, but **can** contain from one and up. We will discuss default colors in a second.  
* color_codes: If you are not satisfied with the default colors or wish to change the rgb color code for a given instance of the mixer, you can do so by specifying colors and their rgb color codes. Normally, you would not want to set this keyword. 
* target: This is not actually used by the color mixer, but can be returned by the `run_cuvette` method such that the output can be in the same format as the output from the `run_cuvette(read_target=True)` method on the LEGO robot. It is smart to keep you *in silico* code as similar to the code you will be running on the robot as possible. Note that the rgb color code input should be of the form (r,g,b). 

Before initializing an instance of the mixer, let us take a look at the default colors of the class. Do this by running the cell below

In [ ]:
SilicoColorMixer.default_color_codes

We have 8 default colors, although you might note that some of them have identical rgb color codes. The color codes are given as lists rather than tuple, e.g., `(249, 242, 30)`. The names of the colors represent the 8 physical colors we have available for the robot. `_u` denotes that the color is purchased from [Urtegården](https://www.urtegaarden.dk/farver). `_t` denotes that it is made by [Tørsleffs](https://www.torsleffs.dk/produkter/farve-gul/).

Now, let us create an instance of the mixer. Although we will be using the default parameters, we write them out for comprehension. Note that since all inputs are keyword arguments, the order does not matter.

In [ ]:
mixer0 = SilicoColorMixer(colors=['red_u', 'green_u', 'blue_t', 'yellow_u'],
                         noise={'colors': 1, 'volume': 0.02, 'measurement': 2},
                         target=None,
                         color_codes={},)

Although the order of the keywords do not matter, the order of colors in the `colors` list does if we want a simple conceptual 1:1 mapping to the robot. The first color in the list, is the color in the first color cartridge from the left and so forth. Once you move to the LEGO robot it is paramount for reuseability of data in one of the last exercises, that the `colors` list matches what is in the color cartridges.

It is, however, not important in this exercise, and you are totally free to experiment with any colors you like.

Now lets go ahead and mix a color like we did in exercise 0. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
input_volumes=[1.0, 0.0, 0.0, 0.0]
output_color = mixer0.run_cuvette(input_volumes)
plt.axis('off')
plt.imshow(np.array([[output_color]], dtype=np.uint8))

Before letting you run free to experiment on your own I would like to showcase the noise. Noise is added to the class to make it more similar to a real system. I will ramp it up just to show you the effect.

In [ ]:
mixer = SilicoColorMixer(noise={'colors': 10, 'volume': 0.1, 'measurement': 10},)

I will then try to mix the same color multiple times - an attempt at equal parts red and yellow - and and show you the difference.

In [ ]:
mixed_colors = []
input_volumes=[1.0, 0.0, 0.0, 1.0]
for n in range (10):
    color = mixer.run_cuvette(input_volumes)
    print(f'Mix {n} =', color)
    mixed_colors.append(color)

plt.axis('off')
plt.imshow(np.array([mixed_colors], dtype=np.uint8))

Now it is your turn to shine. Use the below cells to experiment with the `SilicoColorMixer` and insert more if needed. Make a figure like the one produced by the cell above, but instead of 10 shades of orange, make a rainbow.

If you feel like you are on a roll, try to load a new mixer with different input colors and more or fewer than 4 colors. 

In [ ]:
# my_mixer = SilicoColorMixer(...

# Exercise 2: Difference from target

Next up we will be working with a target color.

In [ ]:
target0 = (220, 141, 50)
mixer_w_target = SilicoColorMixer(noise={'colors': 2, 'volume': 0.02, 'measurement': 2}, target=target0)

We will now do as before but with `read_target=True` in the `run_cuvette` method. This will yield two outputs, which we store in 2 different variables, `color` and `target`.

In [ ]:
input_volumes = [0., 1., 0., 0.]
color, target = mixer_w_target.run_cuvette(input_volumes, read_target=True)
print('Mix =', color, 'Target =', target)
plt.axis('off')
plt.imshow(np.array([[color,target]], dtype=np.uint8))

It is evident that the mixed color and the target color are very different. But **how** different are they?

To answer this **you** will now write a function that takes the color (r, g, b) and target (r, g, b) as inputs and returns some quantitative measure for the difference. To get you started, you can get inspiration from the cell below, which contains a function that takes (r, g, b) tuples (or lists, arrays, etc.) as `input_color1` and `input_color2`and returns the squared value of the red channel of `input_color1`, red$^2$ -- a **very poor** measure for the difference between the two colors across all channels. Try making a function that returns the root summed squared error for each of the r, g, b channels, $\sqrt((\Delta r)^2+(\Delta g)^2+(\Delta b)^2)$

In [ ]:
def red_squared(input_color1, input_color2):
    r_color, g_color, b_color = input_color1[0], input_color1[1], input_color1[2]
    red2 = r_color**2
    return red2
    
red_squared((68, 216, 5), (68, 216, 5))

In [ ]:
# def your_function():
#     ...

Calculate the difference score for a few data points and store them in a list following the example below but using you own "score" function (which is often referred to as loss function) and mixing color combinations of your own choice.

Start by creating an empty list to store data in.

In [ ]:
data = []  # empty list for data. Run only once

Run the below cell a few times with different `input_volumes` to generate some data points. Edit such that you use your own score function.

In [ ]:
# run a handful of times with different color
input_volumes = [0.5, 5, 5.5, 0.5]
color, target = mixer_w_target.run_cuvette(input_volumes, read_target=True)
score = red_squared(color, target)  # replace with your function
data.append([score, list(input_volumes)])
plt.axis('off')
plt.imshow(np.array([[color, target]], dtype=np.uint8))

Take a look at the data you just generated.

In [ ]:
print(data)

Now, lets make a more advanced plot, one showing the calculated difference. First, lets extract the colors we use in the mixer.

In [ ]:
base_colors, base_colors_matrix = mixer_w_target.get_colors()
print(base_colors)
print(base_colors_matrix)

You will now use an advanced plotting function custom made for this course. You can see the Docstring as before by using `Shift`+`Tab`.

In [ ]:
from plot_pie_charts import make_piechart_plot

You should use the data you just generated in the `make_piechart_plot` function. If you do not specify `colors`, it will default to the base colors red, green, blue, and yellow.

In [ ]:
make_piechart_plot(data)

# Exercise 1.3: Optimization

As you now know how to use the color mixer and calculate a difference score, we will use common optimizers find a way to mix the target color. Start by loading in the scipy module. It will install if you do not already have it available. Patience might be required.

In [ ]:
import numpy as np
from IPython import display

try:
    from scipy.optimize import minimize
except:
    !pip3 install scipy --user --upgrade
    from scipy.optimize import minimize

The SciPy minimize function will seek to -- drumroll -- determine that input parameters which minimize the outout of a given function. We fist have to make such a function. For simplicity, make it such that it takes only one input argument (multiple keyword arguments are ok). Before trying with the color mixer, try to optimize a very simple function as the one below. 

In [ ]:
def parabola_4d(func_input):
    """ input is of format [x, y]"""
    return func_input[0]**2 + func_input[1]**2 +func_input[2]**2 +  func_input[3]**2

Set the initial guess, `x0`.

In [ ]:
x0 = [2, -4, 5, 3]

There are countless different optimizers available. The SciPy package has some of the more common once [readily available](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize). No two optimizers are the same and they excel at different tasks. You will use the [conjugate gradient optimizer](https://en.wikipedia.org/wiki/Conjugate_gradient_method) in this exercise but we encourage you to experiment with other optimizers in the SciPy package.

Try experimenting the values of  `eps` and `gtol`.

In [ ]:
res = minimize(parabola_4d, x0, method='CG', options={'disp': True, 'eps': 0.01, 'maxiter': 3, 'gtol': 1}) 
res.x

The last line are the input values that give the minimum output value. We know the correct answer is `[0, 0, 0, 0]`.

Things will, however, not go so smooth for us when trying to optimize the color mixing. We will run issues. What are some of the issues and how do they cause trouble? 

Lets start by removing one of the issues by using the mixer below.

In [ ]:
mixer_opt1 = SilicoColorMixer(target=target0, noise=False)

Write a function where a color is mixing with `mixer_opt1` and a score is calculated. Make sure the function will have its lowest value when the difference between the mixed color and target color is as small as possible. Use the cell below for inspiration. 

In [ ]:
data = []

In [ ]:
def find_color(color_list):
    """Your awesome function"""
    # color, target = mixer_opt1.run_cuvette(...)
    # score = red_squared(color, target)  # Replace with your "score" function
    # data.append(color)
    return score

We halfway solve the second issue by changing optimizer such that we now use `L-BFGS-B` optimizer, which accept bounds. Give the function a starting point and run the minimization. Again, you should try different values of `eps` and `gtol` and possibly also `maxiter` this time.

In [ ]:
x0 = [0.1, 0.1, 0.1, 0.1]

In [ ]:
data=[]

res = minimize(find_color, x0, method='L-BFGS-B',
               bounds = 4*[[0.0, 1.0]], 
               options={'disp': True, 'eps': 0.1, 'maxiter': 3, 'gtol': 0.1})
res

You cannot see how the optimization is progressing. Edit your `find_color` function such that you can collect data and visualize the optimization. You should produce some nice plots. Below are some cells for you to play with. Add more as needed.

At some point you might want to make one of these plots:

In [ ]:
make_piechart_plot(data_pie, colors=base_colors_matrix)
#print(data_pie)

Once you feel like you have some good plots you are ready to try simulating the LEGO robot. The LEGO robot will have to reset or break after every 9 function evalutions due to having filled all mixing cuvettes. We also have to consider noise. Load a mixer with noise.

In [ ]:
mixer_opt2 = SilicoColorMixer(target=target0)

Input your `find_color` function in the cell below, update it to use the new mixer renaming it `find_color2`, and run the cell.

In [ ]:
# def find_color2(color_list):
#     ...

It is possible to force the `find_color` function to raise an error after more than 9 function calls, but it is better to set a maximum number of function evaluations in the optimizer if it allows it. Here `maxfun` (what a name!) is given.

In [ ]:
res = minimize(find_color2, x0, method='L-BFGS-B',
               bounds = 4*[[0.0, 1.0]], 
               options={'disp': True, 'eps': 0.1, 'maxiter': 3, 'gtol': 0.1, 'maxfun': 9})
res

Not surprisingly, 9 function evaluations is not enough to get anywhere. We can even see that the optimizer actually makes more than 9 evaluations. As you will learn later, the LEGO robot control software would have killed the optimization before even one step could be taken. To prevent this from happening, we force the `find_color2` function to wait for our go-ahead every time 9 function evaluations have been made. go ahead finish the function.

In [ ]:
func_calls=[0]

def find_color2(color_list, func_calls=func_calls):  
    color, target = mixer_opt2.run_cuvette(color_list, read_target=True)
    # ...
    # ...
    
    func_calls[0] += 1
    if func_calls[0] % 9 == 0:
        input('Empty cuvettes other than #5. Then press Enter')
        # When LEGO - set the next cuvette to 1 here
    return score

It is not all that pretty, but it works. Try it out. Note that with a noisy mixer, getting `eps` right is especially important. 

In [ ]:
func_calls=[0]

res = minimize(find_color2, x0, method='L-BFGS-B',
               bounds = 4*[[0.0, 1.0]], 
               options={'disp': True, 'eps': 0.1, 'maxiter': 5, 'gtol': 0.1, 'maxfun':100})
res

Now, update the `find_color2` function such that it produces one or more figures for every 9 function evaluations to help you keep track on the progress. Remember, you can always stop the optimization prematurely if you think you are sufficietly converged.

In [ ]:
func_calls=[0]

def find_color2(color_list, func_calls=func_calls):  
    color, target = mixer_opt2.run_cuvette(color_list, read_target=True)
    # ...
    # ...
    
    func_calls[0] += 1
    if func_calls[0] % 9 == 0:
        display.clear_output(wait=True) # This will help you remove figures as you go
        # Your figure plot commands
        input('Have cuvettes other than #5 emptied. Then press Enter')
        # When LEGO - set the next cuvette to 1 here
    return score

In [ ]:
func_calls=[0]

res = minimize(find_color2, x0, method='L-BFGS-B',
               bounds = 4*[[0.0, 1.0]], 
               options={'disp': True, 'eps': 0.1, 'maxiter': 4, 'gtol': 0.1, 'maxfun':60})
res

If you think this has been a piece of cake and you are wondering what you should do next, now would be a great time for you to start experimenting with alternative "score" functions and [other optimizers](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). If you feel like you are already struggling, no worries! You did well to get this far. 

This ends todays exercise. Good job! 